In [77]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [51]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)

In [52]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [53]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [54]:
line = Line([y, z])

In [55]:
eqn_SE1 = P(cubic_new.subs({b:-(c*c+e*f)/c }) )

In [56]:
sing_cubics_SE1 = P(sing_cubics.subs({b:-(c*c+e*f)/c }))

In [57]:
SE1 = Cubic(eqn_SE1, line, sing_cubics_SE1.factor())

In [58]:
SE1.eckardt_points

[(0, 0, 0, 1)]

In [59]:
%time adm_SE1 = SE1.find_admissible_projectivities()

CPU times: user 1.56 s, sys: 181 ms, total: 1.74 s
Wall time: 10.5 s


In [60]:
len(adm_SE1)

576

In [61]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)

CPU times: user 1.03 s, sys: 177 ms, total: 1.21 s
Wall time: 4.66 s


In [62]:
len(simm_SE1)

2

In [63]:
G_SE1 = Group(simm_SE1)

In [64]:
G_SE1.is_group()

True

In [65]:
G_SE1.get_order()

2

In [66]:
G_SE1.is_abelian()

True

In [67]:
G_SE1.get_divisors_of_order()

[1, 2]

In [68]:
G_SE1.get_order_of_elements()

[1, 2]

In [69]:
G_SE1.apply_to(SE1.eckardt_points[0])

[(0, 0, 0, 1), (0, 0, 0, 1)]

In [110]:
SE1.find_admissible_permutations()[1].index(['E1', 'G4', 'E3', 'G6', 'F24'])

95

In [232]:
def find_conditions_for_subfamilies(cubic, sing_cubics, projectivities, simmetries):
    mon = ((x+y+z+t)^3).monomials()
    conditions = []
    sing_cubics_factored = sing_cubics.factor()
    Eck = lcm([el.conditions for el in cubic.tritangent_planes if el.conditions != 0])
    for M in [proj for proj in projectivities if proj not in simmetries]:
        print(projectivities.index(M))
        if projectivities.index(M) == 39:
            continue
        sost = change_coord(M)
        new_cubic = remove_sing_factors(cubic.eqn.subs(sost), sing_cubics_factored)    
        minor = matrix([[new_cubic.coefficient(mn) for mn in mon], [cubic.eqn.coefficient(mn) for mn in mon]]).minors(2)
        minor = [remove_sing_factors(el, sing_cubics_factored) for el in minor if el !=0]
        ideale = cubic.P.ideal(minor)
        del_d_ideale = ideale.elimination_ideal([c])
        prim_deco = del_d_ideale.radical().primary_decomposition()
        for ideal in prim_deco:
            prim_deco2 = (ideale + ideal).primary_decomposition()
            for ideal2 in prim_deco2:
                gener = tuple([remove_sing_factors(gen, sing_cubics_factored) for gen in ideal2.gens()])
                if 1 not in gener:
                    flag = True
                    for cond in gener:
                        if cond.divides(Eck):
                            flag = False
                            break
                    if flag is True:
                        conditions.append(gener)                        
    return list(set(conditions))

In [233]:
M = adm_SE1[39]

In [234]:
conds = find_conditions_for_subfamilies(SE1, sing_cubics_SE1, adm_SE1, simm_SE1)

0
1
2
3
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


KeyboardInterrupt: 

In [170]:
SE1.cl_lines['E1']

(0, 0, 1, 0, 0, 0) [y, z]

In [169]:
SE1.cl_lines['E1'].apply_proj(M)

(0, 0, 1, 0, 0, 0) [8*y*c^9 + 8*y*c^8*d + 8*y*c^8*e + 8*y*c^7*d*e + 16*y*c^8*f + 16*y*c^7*d*f + 8*y*c^7*e*f + 16*y*c^6*d*e*f - 8*y*c^6*e^2*f + 8*y*c^7*f^2 + 8*y*c^6*d*f^2 - 8*y*c^6*e*f^2 + 8*y*c^5*d*e*f^2 - 16*y*c^5*e^2*f^2 - 8*y*c^5*e*f^3 - 8*y*c^4*e^2*f^3, 2*y*c^9 - 2*z*c^9 - 2*y*c^8*d + 2*z*c^8*d + 4*y*c^8*e - 4*z*c^8*e - 4*y*c^7*d*e + 4*z*c^7*d*e + 2*y*c^7*e^2 - 2*z*c^7*e^2 - 2*y*c^6*d*e^2 + 2*z*c^6*d*e^2 + 14*y*c^8*f - 6*z*c^8*f + 10*y*c^7*d*f - 2*z*c^7*d*f + 22*y*c^7*e*f - 14*z*c^7*e*f + 12*y*c^6*d*e*f - 4*z*c^6*d*e*f + 10*y*c^6*e^2*f - 10*z*c^6*e^2*f + 2*y*c^5*d*e^2*f - 2*z*c^5*d*e^2*f + 2*y*c^5*e^3*f - 2*z*c^5*e^3*f + 14*y*c^7*f^2 + 2*z*c^7*f^2 + 18*y*c^6*d*f^2 - 2*z*c^6*d*f^2 + 2*y*c^6*e*f^2 + 6*z*c^6*e*f^2 + 20*y*c^5*d*e*f^2 - 4*z*c^5*d*e*f^2 - 14*y*c^5*e^2*f^2 + 6*z*c^5*e^2*f^2 + 2*y*c^4*d*e^2*f^2 - 2*z*c^4*d*e^2*f^2 - 2*y*c^4*e^3*f^2 + 2*z*c^4*e^3*f^2 + 2*y*c^6*f^3 + 6*z*c^6*f^3 + 6*y*c^5*d*f^3 + 2*z*c^5*d*f^3 - 22*y*c^5*e*f^3 + 14*z*c^5*e*f^3 + 4*y*c^4*d*e*f^3 + 4*z*c^4*d*

In [172]:
SE1.cl_lines['G4']

(0, 0, 0, 0, 0, 1) [y, x]

In [184]:
SE1.cl_lines['G4'].apply_proj(M).plucker

(0, 0, -c^5 + c^4*d - c^4*e + c^3*d*e - 2*c^4*f - 2*c^3*d*f - 3*c^3*e*f - 2*c^2*d*e*f - c^2*e^2*f + 3*c^3*f^2 + c^2*d*f^2 + 5*c^2*e*f^2 + c*d*e*f^2 + 2*c*e^2*f^2 - c*e*f^3 - e^2*f^3, 0, 0, 2*c^5 + 6*c^4*d - 2*c^4*e + 2*c^3*d*e + 8*c^3*d*f - 14*c^3*e*f - 2*c^2*e^2*f - 2*c^3*f^2 + 2*c^2*d*f^2 - 14*c^2*e*f^2 - 2*c*d*e*f^2 - 2*c*e*f^3 + 2*e^2*f^3)

In [186]:
pol = matrix([SE1.cl_lines['F14'].plucker, SE1.cl_lines['G4'].apply_proj(M).plucker]).minors(2)[-4]//(c+f)

In [173]:
SE1.cl_lines['F14']

(0, 0, c^2 + c*d - e*f, 0, 0, -c^2 - c*e - c*f - e*f) [y, -x*c^2 - z*c^2 - z*c*d - x*c*e - x*c*f - x*e*f + z*e*f]

In [175]:
SE1.cl_lines['F14'].apply_proj(M)

(0, 0, 0, 0, 0, 1) [8*y*c^9 + 8*y*c^8*d + 8*y*c^8*e + 8*y*c^7*d*e + 16*y*c^8*f + 16*y*c^7*d*f + 8*y*c^7*e*f + 16*y*c^6*d*e*f - 8*y*c^6*e^2*f + 8*y*c^7*f^2 + 8*y*c^6*d*f^2 - 8*y*c^6*e*f^2 + 8*y*c^5*d*e*f^2 - 16*y*c^5*e^2*f^2 - 8*y*c^5*e*f^3 - 8*y*c^4*e^2*f^3, -4*x*c^11 - 16*x*c^10*d + 8*y*c^10*d - 12*x*c^9*d^2 + 8*y*c^9*d^2 - 8*y*c^10*e - 16*x*c^9*d*e - 16*x*c^8*d^2*e + 8*y*c^8*d^2*e + 4*x*c^9*e^2 - 8*y*c^9*e^2 - 8*y*c^8*d*e^2 - 4*x*c^7*d^2*e^2 - 4*x*c^10*f - 8*y*c^10*f - 32*x*c^9*d*f + 8*y*c^9*d*f - 28*x*c^8*d^2*f + 16*y*c^8*d^2*f + 32*x*c^9*e*f - 40*y*c^9*e*f + 8*x*c^8*d*e*f - 32*y*c^8*d*e*f - 32*x*c^7*d^2*e*f + 16*y*c^7*d^2*e*f + 36*x*c^8*e^2*f - 24*y*c^8*e^2*f + 48*x*c^7*d*e^2*f - 40*y*c^7*d*e^2*f - 4*x*c^6*d^2*e^2*f + 8*y*c^7*e^3*f + 8*x*c^6*d*e^3*f + 4*x*c^9*f^2 - 16*y*c^9*f^2 - 16*x*c^8*d*f^2 - 8*y*c^8*d*f^2 - 20*x*c^7*d^2*f^2 + 8*y*c^7*d^2*f^2 + 64*x*c^8*e*f^2 - 48*y*c^8*e*f^2 + 72*x*c^7*d*e*f^2 - 64*y*c^7*d*e*f^2 - 16*x*c^6*d^2*e*f^2 + 8*y*c^6*d^2*e*f^2 + 32*x*c^7*e^2*f^2 + 96*

In [190]:
conds[-1]

48*c^6 + 128*c^5*d + 16*c^4*d^2 - 16*c^5*e + 64*c^4*d*e + 16*c^3*d^2*e - 16*c^5*f + 64*c^4*d*f + 16*c^3*d^2*f - 208*c^4*e*f - 32*c^3*d*e*f + 16*c^2*d^2*e*f - 64*c^3*e^2*f - 32*c^2*d*e^2*f - 64*c^3*e*f^2 - 32*c^2*d*e*f^2 + 16*c^2*e^2*f^2 - 32*c*d*e^2*f^2 + 16*c*e^3*f^2 + 16*c*e^2*f^3 + 16*e^3*f^3

In [191]:
pol

c^6 + 9*c^5*d + 6*c^4*d^2 - 4*c^5*e + 2*c^4*d*e + 2*c^3*d^2*e - c^4*e^2 + c^3*d*e^2 - 4*c^5*f - 2*c^4*d*f + 2*c^3*d^2*f - 19*c^4*e*f - 24*c^3*d*e*f - 2*c^2*d^2*e*f - 4*c^3*e^2*f - 6*c^2*d*e^2*f - c^2*e^3*f + 3*c^4*f^2 + c^3*d*f^2 + 8*c^3*e*f^2 - 2*c^2*d*e*f^2 + 21*c^2*e^2*f^2 + 5*c*d*e^2*f^2 + 4*c*e^3*f^2 - c^2*e*f^3 - 3*e^3*f^3

In [193]:
pol in P.ideal(conds)

True

In [194]:
for el in conds:
    print(el.degrees())

(0, 0, 0, 0, 0, 10, 2, 5, 5, 0, 0)
(0, 0, 0, 0, 0, 10, 3, 5, 6, 0, 0)
(0, 0, 0, 0, 0, 11, 3, 5, 6, 0, 0)
(0, 0, 0, 0, 0, 10, 2, 5, 5, 0, 0)
(0, 0, 0, 0, 0, 9, 3, 5, 6, 0, 0)
(0, 0, 0, 0, 0, 11, 3, 5, 6, 0, 0)
(0, 0, 0, 0, 0, 10, 2, 5, 5, 0, 0)
(0, 0, 0, 0, 0, 10, 2, 5, 5, 0, 0)
(0, 0, 0, 0, 0, 10, 3, 5, 6, 0, 0)
(0, 0, 0, 0, 0, 11, 3, 5, 6, 0, 0)
(0, 0, 0, 0, 0, 10, 2, 5, 5, 0, 0)
(0, 0, 0, 0, 0, 9, 3, 5, 6, 0, 0)
(0, 0, 0, 0, 0, 11, 3, 5, 6, 0, 0)
(0, 0, 0, 0, 0, 10, 2, 5, 5, 0, 0)
(0, 0, 0, 0, 0, 10, 3, 5, 5, 0, 0)
(0, 0, 0, 0, 0, 6, 2, 3, 3, 0, 0)
(0, 0, 0, 0, 0, 10, 2, 5, 5, 0, 0)
(0, 0, 0, 0, 0, 9, 3, 4, 5, 0, 0)
(0, 0, 0, 0, 0, 10, 3, 5, 5, 0, 0)
(0, 0, 0, 0, 0, 10, 2, 5, 5, 0, 0)
(0, 0, 0, 0, 0, 11, 3, 5, 6, 0, 0)
(0, 0, 0, 0, 0, 11, 3, 5, 6, 0, 0)
(0, 0, 0, 0, 0, 9, 3, 5, 6, 0, 0)
(0, 0, 0, 0, 0, 10, 3, 5, 5, 0, 0)
(0, 0, 0, 0, 0, 10, 3, 5, 6, 0, 0)
(0, 0, 0, 0, 0, 11, 3, 5, 6, 0, 0)
(0, 0, 0, 0, 0, 10, 3, 5, 6, 0, 0)
(0, 0, 0, 0, 0, 6, 2, 3, 3, 0, 0)
(0, 0, 0, 0, 0, 6, 2, 3, 3

In [209]:
ideale = P.ideal(conds)

In [201]:
polyn = conds[-4]

In [203]:
polynd = polyn.polynomial(d)

In [225]:
ideale.elimination_ideal([c]).radical().primary_decomposition()

[Ideal (8*d*e + 27*e^2 + 8*d*f - 38*e*f + 19*f^2, 8*d^2 + 13*e^2 - 16*d*f + 54*e*f + 21*f^2, 119*e^3 - 259*e^2*f - 96*d*f^2 + 425*e*f^2 - 109*f^3) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field,
 Ideal (d + 5*e - 3*f, 5*e^2 - 4*e*f + f^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field,
 Ideal (d + e - f) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field,
 Ideal (d - e + f) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field,
 Ideal (e - f) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field,
 Ideal (e) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field,
 Ideal (f) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field]

In [223]:
(ideale+_[0]).radical().primary_decomposition()

[Ideal (f, e, c) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field,
 Ideal (7*c + d - e - 2*f, 8*d*e + 27*e^2 + 8*d*f - 38*e*f + 19*f^2, 8*d^2 + 13*e^2 - 16*d*f + 54*e*f + 21*f^2, 119*e^3 - 259*e^2*f - 96*d*f^2 + 425*e*f^2 - 109*f^3) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field]

In [127]:
Q = PolynomialRing(QQ, [c,d,e,f])

In [132]:
idea = Q.ideal(conds[0])

In [ ]:
idea.triangular_decomposition()